In [6]:
import numpy as np

In [7]:
P = 18
K = 4
batch_size = P * K
embedding_size = 15

In [8]:
a = np.arange(5)
b = np.arange(5) * 10
print(a, b)
b = b[None, :]
print(a.shape, b.shape)
    

[0 1 2 3 4] [ 0 10 20 30 40]
(5,) (1, 5)


In [9]:
print(b.T)
a + b.T

[[ 0]
 [10]
 [20]
 [30]
 [40]]


array([[ 0,  1,  2,  3,  4],
       [10, 11, 12, 13, 14],
       [20, 21, 22, 23, 24],
       [30, 31, 32, 33, 34],
       [40, 41, 42, 43, 44]])

In [10]:
x = np.arange(batch_size * embedding_size)

In [11]:
x.shape = (batch_size, embedding_size)
print(x)

[[   0    1    2 ...,   12   13   14]
 [  15   16   17 ...,   27   28   29]
 [  30   31   32 ...,   42   43   44]
 ..., 
 [1035 1036 1037 ..., 1047 1048 1049]
 [1050 1051 1052 ..., 1062 1063 1064]
 [1065 1066 1067 ..., 1077 1078 1079]]


broadcasting:
operation is executed along dimension 1

In [12]:
# [x, y ,z]
# shape = 1 x 8 x 15
x_a = x[None, : ,:]
# shape = 8 x 1 x 15
x_b = x[:, None, :]
print(x_a.shape)
print(x_b.shape)

(1, 72, 15)
(72, 1, 15)


Calculate cdist the distance between the vectors, should return batch_size x batch_size

In [255]:
cdist = x_b - x_a
print(cdist.shape)
print(cdist)


(72, 72, 15)
[[[    0     0     0 ...,     0     0     0]
  [  -15   -15   -15 ...,   -15   -15   -15]
  [  -30   -30   -30 ...,   -30   -30   -30]
  ..., 
  [-1035 -1035 -1035 ..., -1035 -1035 -1035]
  [-1050 -1050 -1050 ..., -1050 -1050 -1050]
  [-1065 -1065 -1065 ..., -1065 -1065 -1065]]

 [[   15    15    15 ...,    15    15    15]
  [    0     0     0 ...,     0     0     0]
  [  -15   -15   -15 ...,   -15   -15   -15]
  ..., 
  [-1020 -1020 -1020 ..., -1020 -1020 -1020]
  [-1035 -1035 -1035 ..., -1035 -1035 -1035]
  [-1050 -1050 -1050 ..., -1050 -1050 -1050]]

 [[   30    30    30 ...,    30    30    30]
  [   15    15    15 ...,    15    15    15]
  [    0     0     0 ...,     0     0     0]
  ..., 
  [-1005 -1005 -1005 ..., -1005 -1005 -1005]
  [-1020 -1020 -1020 ..., -1020 -1020 -1020]
  [-1035 -1035 -1035 ..., -1035 -1035 -1035]]

 ..., 
 [[ 1035  1035  1035 ...,  1035  1035  1035]
  [ 1020  1020  1020 ...,  1020  1020  1020]
  [ 1005  1005  1005 ...,  1005  1005  1005]
  ...

In [256]:
cdist = np.square(cdist)
cdist = np.sum(cdist, axis=2)
cdist = np.sqrt(cdist)
print(cdist.shape)
print(cdist)

(72, 72)
[[    0.            58.09475019   116.18950039 ...,  4008.53776332
   4066.63251352  4124.72726371]
 [   58.09475019     0.            58.09475019 ...,  3950.44301313
   4008.53776332  4066.63251352]
 [  116.18950039    58.09475019     0.         ...,  3892.34826294
   3950.44301313  4008.53776332]
 ..., 
 [ 4008.53776332  3950.44301313  3892.34826294 ...,     0.            58.09475019
    116.18950039]
 [ 4066.63251352  4008.53776332  3950.44301313 ...,    58.09475019     0.
     58.09475019]
 [ 4124.72726371  4066.63251352  4008.53776332 ...,   116.18950039
     58.09475019     0.        ]]


create mask that selects K images of the same person in each row

In [257]:
import time

In [55]:
pids = np.array([1, 1, 15, 1, 15, 15, 15, 15])
pids[None,:] == pids[:,None]

array([[ True,  True, False,  True, False, False, False, False],
       [ True,  True, False,  True, False, False, False, False],
       [False, False,  True, False,  True,  True,  True,  True],
       [ True,  True, False,  True, False, False, False, False],
       [False, False,  True, False,  True,  True,  True,  True],
       [False, False,  True, False,  True,  True,  True,  True],
       [False, False,  True, False,  True,  True,  True,  True],
       [False, False,  True, False,  True,  True,  True,  True]], dtype=bool)

In [45]:
def create_mask():
    mask = np.eye(P, dtype=int)
    return mask.repeat(K, axis=0).repeat(K, axis=1)


In [273]:
%timeit create_mask()
print(create_mask())


29.4 µs ± 1.63 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
[[1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]


In [132]:
def create_mask2():
    return np.kron(np.eye(P), np.ones((K, K)))

In [133]:
%timeit create_mask2()
print(create_mask2())

67.5 µs ± 1.35 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
[[ 1.  1.  1. ...,  0.  0.  0.]
 [ 1.  1.  1. ...,  0.  0.  0.]
 [ 1.  1.  1. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  1.  1.  1.]
 [ 0.  0.  0. ...,  1.  1.  1.]
 [ 0.  0.  0. ...,  1.  1.  1.]]


In [134]:
def create_mask3():
    # faster for bigger mattrices
    from numpy.lib.stride_tricks import as_strided
    a = np.eye(P, dtype=bool)
    r, c = a.shape                                    # number of rows/columns
    rs, cs = a.strides                                # row/column strides 
    x = as_strided(a, (r, K, c, K), (rs, 0, cs, 0)) # view a as larger 4D array
    return x.reshape(r*K, c*K)

In [135]:
%timeit create_mask3()
print(create_mask3())

28.5 µs ± 889 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
[[ True  True  True ..., False False False]
 [ True  True  True ..., False False False]
 [ True  True  True ..., False False False]
 ..., 
 [False False False ...,  True  True  True]
 [False False False ...,  True  True  True]
 [False False False ...,  True  True  True]]


In [259]:
mask = create_mask()
print(mask)

[[ True  True  True ..., False False False]
 [ True  True  True ..., False False False]
 [ True  True  True ..., False False False]
 ..., 
 [False False False ...,  True  True  True]
 [False False False ...,  True  True  True]
 [False False False ...,  True  True  True]]


In [260]:
print(cdist.shape)
print(mask.shape)
print(cdist)

(72, 72)
(72, 72)
[[    0.            58.09475019   116.18950039 ...,  4008.53776332
   4066.63251352  4124.72726371]
 [   58.09475019     0.            58.09475019 ...,  3950.44301313
   4008.53776332  4066.63251352]
 [  116.18950039    58.09475019     0.         ...,  3892.34826294
   3950.44301313  4008.53776332]
 ..., 
 [ 4008.53776332  3950.44301313  3892.34826294 ...,     0.            58.09475019
    116.18950039]
 [ 4066.63251352  4008.53776332  3950.44301313 ...,    58.09475019     0.
     58.09475019]
 [ 4124.72726371  4066.63251352  4008.53776332 ...,   116.18950039
     58.09475019     0.        ]]


In [261]:
np.amax(cdist, axis=1)

array([ 4124.72726371,  4066.63251352,  4008.53776332,  3950.44301313,
        3892.34826294,  3834.25351275,  3776.15876255,  3718.06401236,
        3659.96926217,  3601.87451197,  3543.77976178,  3485.68501159,
        3427.59026139,  3369.4955112 ,  3311.40076101,  3253.30601081,
        3195.21126062,  3137.11651043,  3079.02176023,  3020.92701004,
        2962.83225985,  2904.73750966,  2846.64275946,  2788.54800927,
        2730.45325908,  2672.35850888,  2614.26375869,  2556.1690085 ,
        2498.0742583 ,  2439.97950811,  2381.88475792,  2323.79000772,
        2265.69525753,  2207.60050734,  2149.50575715,  2091.41100695,
        2091.41100695,  2149.50575715,  2207.60050734,  2265.69525753,
        2323.79000772,  2381.88475792,  2439.97950811,  2498.0742583 ,
        2556.1690085 ,  2614.26375869,  2672.35850888,  2730.45325908,
        2788.54800927,  2846.64275946,  2904.73750966,  2962.83225985,
        3020.92701004,  3079.02176023,  3137.11651043,  3195.21126062,
      

In [262]:
np.amin(cdist, axis=1)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [263]:
m = 0.3

In [271]:
def create_cdist(cdist):
    # this seems to be slightly faster
    cdist_max = cdist.copy()
    cdist_max[np.logical_not(mask)] = 0
    cdist_min = cdist.copy()
    cdist_min[mask] = 0
    return cdist_min, cdist_max
    
    

In [274]:
def create_cdist2(cdist):
    max_cdist = cdist * mask
    min_cdist = cdist * np.logical_not(mask)
    return min_cdist, max_cdist

In [275]:
#cdist_min1, cdist_max1 = create_cdist(cdist)
%timeit cdist_min2, cdist_max2 = create_cdist2(cdist)
print(cdist_min2)
print(np.array_equal(cdist_min1, cdist_min2))
print(np.array_equal(cdist_max1, cdist_max2))


17.9 µs ± 173 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
[[    0.             0.             0.         ...,  4008.53776332
   4066.63251352  4124.72726371]
 [    0.             0.             0.         ...,  3950.44301313
   4008.53776332  4066.63251352]
 [    0.             0.             0.         ...,  3892.34826294
   3950.44301313  4008.53776332]
 ..., 
 [ 4008.53776332  3950.44301313  3892.34826294 ...,     0.             0.
      0.        ]
 [ 4066.63251352  4008.53776332  3950.44301313 ...,     0.             0.
      0.        ]
 [ 4124.72726371  4066.63251352  4008.53776332 ...,     0.             0.
      0.        ]]
False
False


In [276]:
print(cdist_min1)

[[   0.           58.09475019  116.18950039 ...,    0.            0.            0.        ]
 [  58.09475019    0.           58.09475019 ...,    0.            0.            0.        ]
 [ 116.18950039   58.09475019    0.         ...,    0.            0.            0.        ]
 ..., 
 [   0.            0.            0.         ...,    0.           58.09475019
   116.18950039]
 [   0.            0.            0.         ...,   58.09475019    0.
    58.09475019]
 [   0.            0.            0.         ...,  116.18950039
    58.09475019    0.        ]]


In [277]:
loss = np.amax(cdist_max, axis=1) - np.amin(cdist_min, axis=1) + m
print(loss)
loss[loss < 0] = 0.0
loss = 1/(batch_size) * np.sum(loss)

[ 174.58425058  116.48950039  116.48950039  174.58425058]


In [278]:
print(loss)

8.08538197127


In [27]:
import torch
from torch.autograd import Variable

In [28]:
print(x)

[[   0    1    2 ...,   12   13   14]
 [  15   16   17 ...,   27   28   29]
 [  30   31   32 ...,   42   43   44]
 ..., 
 [1035 1036 1037 ..., 1047 1048 1049]
 [1050 1051 1052 ..., 1062 1063 1064]
 [1065 1066 1067 ..., 1077 1078 1079]]


In [29]:
x_t = Variable(torch.from_numpy(x))
print(x_t)

Variable containing:
    0     1     2  ...     12    13    14
   15    16    17  ...     27    28    29
   30    31    32  ...     42    43    44
       ...          ⋱          ...       
 1035  1036  1037  ...   1047  1048  1049
 1050  1051  1052  ...   1062  1063  1064
 1065  1066  1067  ...   1077  1078  1079
[torch.LongTensor of size 72x15]



In [32]:
# shape = 1 x 8 x 15
x_a = x_t[None, : ,:]
# shape = 8 x 1 x 15
x_b = x_t[:, None, :]
print(x_a.data.shape)
print(x_b.data.shape)

torch.Size([1, 72, 15])
torch.Size([72, 1, 15])


In [42]:
cdist = x_b - x_a
print(cdist.data.shape)


torch.Size([72, 72, 15])


In [43]:
cdist = torch.mul(cdist, cdist)
cdist = torch.sum(cdist, dim=2)
print(cdist.data.shape)

torch.Size([72, 72])


In [51]:
def create_mask():
    mask = np.eye(P, dtype=int)
    return mask.repeat(K, axis=0).repeat(K, axis=1)
mask = create_mask()

In [52]:
print(mask)
print(np.logical_not(mask))t

[[1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]
[[False False False ...,  True  True  True]
 [False False False ...,  True  True  True]
 [False False False ...,  True  True  True]
 ..., 
 [ True  True  True ..., False False False]
 [ True  True  True ..., False False False]
 [ True  True  True ..., False False False]]


In [68]:
mask = create_mask()
print(mask)
print(1-mask)
mask_max = Variable(torch.from_numpy(mask))
mask_min = (1-mask).astype(np.byte)
print(mask_min)
mask_min = Variable(torch.from_numpy(mask_min))

[[1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]
[[0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 ..., 
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]]
[[0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 ..., 
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]]


RuntimeError: can't convert a given np.ndarray to a tensor - it has an invalid type. The only supported types are: double, float, int64, int32, and uint8.

In [71]:
target = np.arange(P).repeat(K)
print(target)
target = Variable(torch.from_numpy(target))

[ 0  0  0  0  1  1  1  1  2  2  2  2  3  3  3  3  4  4  4  4  5  5  5  5  6
  6  6  6  7  7  7  7  8  8  8  8  9  9  9  9 10 10 10 10 11 11 11 11 12 12
 12 12 13 13 13 13 14 14 14 14 15 15 15 15 16 16 16 16 17 17 17 17]


In [85]:
mask_max = target[None,:] == target[:,None]
print(mask)
mask_min = 1 - mask_max.data
print(mask_min)

Variable containing:
    1     1     1  ...      0     0     0
    1     1     1  ...      0     0     0
    1     1     1  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      1     1     1
    0     0     0  ...      1     1     1
    0     0     0  ...      1     1     1
[torch.ByteTensor of size 72x72]


    0     0     0  ...      1     1     1
    0     0     0  ...      1     1     1
    0     0     0  ...      1     1     1
       ...          ⋱          ...       
    1     1     1  ...      0     0     0
    1     1     1  ...      0     0     0
    1     1     1  ...      0     0     0
[torch.ByteTensor of size 72x72]



In [94]:
max_cdist = torch.masked_select(cdist, mask_max).view(K, batch_size)
min_cdist = torch.masked_select(cdist, mask_min).view(batch_size - K, batch_size)

In [97]:
print(max_cdist.size())
print(min_cdist.size())

torch.Size([4, 72])
torch.Size([68, 72])


In [107]:
loss_max = torch.max(max_cdist, dim=0)[0] # max returns tuples with the indexes?
loss_min = torch.min(min_cdist, dim=0)[0]

In [108]:
print(loss_max)
print(loss_min)

Variable containing:
 13500
  3375
 13500
 30375
 30375
 13500
  3375
 13500
 13500
  3375
 13500
 30375
 30375
 13500
  3375
 13500
 13500
  3375
 13500
 30375
 30375
 13500
  3375
 13500
 13500
  3375
 13500
 30375
 30375
 13500
  3375
 13500
 13500
  3375
 13500
 30375
 30375
 13500
  3375
 13500
 13500
  3375
 13500
 30375
 30375
 13500
  3375
 13500
 13500
  3375
 13500
 30375
 30375
 13500
  3375
 13500
 13500
  3375
 13500
 30375
 30375
 13500
  3375
 13500
 13500
  3375
 13500
 30375
 30375
 13500
  3375
 13500
[torch.LongTensor of size 72]

Variable containing:
  3375
 13500
 30375
 30375
 13500
 30375
 30375
 13500
 30375
 30375
 13500
  3375
  3375
 13500
 30375
 30375
 13500
 30375
 30375
 13500
 30375
 30375
 13500
  3375
  3375
 13500
 30375
 30375
 13500
 30375
 30375
 13500
 30375
 30375
 13500
  3375
  3375
 13500
 30375
 30375
 13500
 30375
 30375
 13500
 30375
 30375
 13500
  3375
  3375
 13500
 30375
 30375
 13500
 30375
 30375
 13500
 30375
 30375
 13500
  3375
  3

In [113]:
m = 3

In [116]:
loss = Variable(torch.LongTensor(m)) + loss_max - loss_min

RuntimeError: inconsistent tensor size, expected r_ [3], t [3] and src [72] to have the same number of elements, but got 3, 3 and 72 elements respectively at /pytorch/torch/lib/TH/generic/THTensorMath.c:887